In [ ]:
!pip install Pillow

In [ ]:
%%markdown
## Imports

In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
%%markdown
## Establish connections

In [ ]:
# Database connection parameters
DB_HOST = "localhost"
DB_PORT = "5433"
DB_NAME = "test"
DB_USER = "postgres"
DB_PASSWORD = "postgres"

In [ ]:
def connect_psycopg2():
    """Establish a connection to PostgreSQL using psycopg2."""
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        print("Connected to PostgreSQL successfully using psycopg2!")
        return conn
    except Exception as e:
        print("Error connecting to PostgreSQL:", e)
        raise

In [ ]:
conn = connect_psycopg2()
cur = conn.cursor()

In [ ]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS employees (
        id SERIAL PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        age INTEGER,
        department VARCHAR(50)
    );
    """
cur.execute(create_table_query)
conn.commit()

In [ ]:
%%markdown
## Create Table

In [ ]:
insert_query = "INSERT INTO employees (name, age, department) VALUES (%s, %s, %s);"
employees_data = [
    ('Alice', 30, 'HR'),
    ('Bob', 25, 'Engineering'),
    ('Charlie', 35, 'Finance')
]
cur.executemany(insert_query, employees_data)
conn.commit()

In [ ]:
%%markdown
## Update

In [ ]:
update_query = "UPDATE employees SET department = %s WHERE name = %s;"
cur.execute(update_query, ('Marketing', 'Alice'))
conn.commit()

In [ ]:
%%markdown
## Delete

In [ ]:
delete_query = "DELETE FROM employees WHERE name = %s;"
cur.execute(delete_query, ('Charlie',))
conn.commit()

In [ ]:
%%markdown
## PANDAS SELECT EXAMPLE

In [ ]:
query = "SELECT * FROM employees;"
# Using pandas read_sql_query with the psycopg2 connection
df = pd.read_sql_query(query, conn)
print("\nData fetched using pandas:")
print(df)

In [ ]:
%%markdown
## Closing connection

In [ ]:
cur.close()
conn.close()

In [ ]:
%%markdown
## Handling warning and using SQLAlchemy

In [ ]:
engine_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(engine_str)
query = "SELECT * FROM employees;"
df = pd.read_sql(query, engine)
print(df)

In [ ]:
%%markdown
## Sample Transformation and ETL

In [ ]:
df_transformed = df.copy()
df_transformed['age_in_5_years'] = df_transformed['age'] + 5
df_transformed['department_id'] = df_transformed['department'].apply(lambda x: 1 if x == 'Engineering' else 0)
df_transformed.to_sql('employees_staging', engine, if_exists='replace', index=False)

In [ ]:
%%markdown
## Let us see a bit larger picture

In [ ]:
from IPython.display import display
from PIL import Image

image_path = "simple_python_usecase.png"
img = Image.open(image_path)
display(img)

In [ ]:
%%markdown
## References
1. https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html
2. https://medium.com/oracledevs/using-the-development-branch-of-sqlalchemy-2-0-with-python-oracledb-d6e89090899c